In [20]:
1

1

In [21]:
import pdfplumber
import json
import re

def extract_ttas_tables(pdf_path):
    all_content = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            # 提取表格數據
            tables = page.extract_tables()
            # 提取當頁純文字（用於獲取標題與備註）
            text = page.extract_text()
            all_content.append({"text": text, "tables": tables})
    return all_content

In [33]:
# file= extract_ttas_tables(r"C:\Users\ygz08\Work\TTAS\code\rag_files\急診五級檢傷分類基準修正版-環境.pdf")
# file= extract_ttas_tables(r"C:\Users\ygz08\Work\TTAS\code\rag_files\急診五級檢傷分類基準修正版-外傷.pdf")
# file = extract_ttas_tables(r"C:\Users\ygz08\Work\TTAS\code\rag_files\急診五級檢傷分類基準修正版-兒童.pdf")
# file = extract_ttas_tables(r"C:\Users\ygz08\Work\TTAS\code\rag_files\急診五級檢傷分類基準修正版-兒童(第二次修正).pdf")
file = extract_ttas_tables(r"C:\Users\ygz08\Work\TTAS\code\rag_files\急診五級檢傷分類基準修正版-成人非外傷 (1).pdf")

In [4]:
# file[0] # page 0
# print(file[0]["tables"])# page 0 的table
# print(file[0]["tables"][0][0]) # 第一row #標題
# print(file[0]["tables"][0][1]) # 第二row ，第一筆正式資料。
# print(file[0]["tables"][0][1][0]) # 大分類名稱
# print(file[0]["tables"][0][1][1]) # 標準主訴名稱 != None 代表在同一個主訴內 
# print(file[0]["tables"][0][1][2]) #判定依據中文名稱
# print(file[0]["tables"][0][1][3]) # TTAS 分級

In [37]:
# 輔助函式：把原本重複的 if-elif 統整邏輯包起來
def format_content(l1, l2, l3, l4, l5):
    items = []
    if l1: items.append(f"- 一級 (復甦急救)：{l1}")
    if l2: items.append(f"- 二級 (危急)：{l2}")
    if l3: items.append(f"- 三級 (緊急)：{l3}")
    if l4: items.append(f"- 四級 (次緊急)：{l4}")
    if l5: items.append(f"- 五級 (非緊急)：{l5}")
    return "\n".join(items)
    
level_one, level_two, level_three, level_four, level_five = "", "", "", "", ""
adjust_str = ""
var = ""
all_results = [] # TODO:1 準備裝所有的 dict1
list_var = []
dict1 = None # 先初始化為 None

# 檔案層級資訊 (這部分維持不變)
category = file[0]["text"].split('\n')[0]
target = "成人"#"兒童"#"成人/兒童" 

standard_name = 1
determined_name = 2
TTAS_level = 3
adjust = 4

for page in range(len(file)):
    if not file[page].get("tables") or len(file[page]["tables"]) == 0:
        #print(f"第 {page+1} 頁沒有偵測到表格，跳過。")
        continue
    for row in range(1, len(file[page]["tables"][0])):
        
        # 1. 偵測大分類 (Sub-category)
        if file[page]["tables"][0][row][0] is not None and file[page]["tables"][0][row][0] != '':
            top_category_name = file[page]["tables"][0][row][0]

        # 2. 偵測到「新的標準主訴」開始
        current_standard_val = file[page]["tables"][0][row][standard_name]
        if current_standard_val is not None and current_standard_val != '':
            
            # ---【重要】在開始新的之前，先把「上一個」統整並存起來 ---
            if dict1 is not None:
                # 這裡執行你的最後統整邏輯
                dict1["content"] = format_content(level_one, level_two, level_three, level_four, level_five)
                dict1["remarks"] = adjust_str
                all_results.append(dict1) # 存入清單
                formatted_json = json.dumps(dict1, indent=4, ensure_ascii=False)
                print(f'{var} = {formatted_json}')  
                # print(json.dumps(dict1, indent=4, ensure_ascii=False)) # 若要即時看也可以

            # --- 初始化「下一個」主訴的資料 ---
            now_name_raw = current_standard_val
            match = re.match(r"([A-Z]\d+)", str(now_name_raw)) # 確保 now_name_raw 是字串

            if match:
                var = match.group(0)
            else:
                # 如果沒匹配到，給它一個預設值或是跳過
                var = "Unknown" 
                print(f"警告：字串 '{now_name_raw}' 無法匹配編號規範")
            now_name = re.sub(rf"^{var}", "", now_name_raw).strip()
            now_name = now_name.replace('\n', '')
            top_category_name = top_category_name.replace('\n', '')
            now_name = now_name.replace('\n', '')

            dict1 = {
                "metadata": {
                    "category": category,
                    "sub_category": top_category_name,
                    "target": target,
                },
                "query_text": now_name,
                "content": "",
                "remarks": ""
            }
            list_var.append(var)
            # 重置累積字串
            level_one, level_two, level_three, level_four, level_five = "", "", "", "", ""
            adjust_str = "可調節分級項目 : "

        # 3. 累積每一行的資料 (不管有沒有新的主訴名稱，每一行都要跑這段)
        subtitle = file[page]["tables"][0][row][determined_name]
        subtitle = subtitle.replace('\n', '')
        ttas = str(file[page]["tables"][0][row][TTAS_level])
        
        if ttas == "1": level_one += subtitle + "。"
        elif ttas == "2": level_two += subtitle + "。"
        elif ttas == "3": level_three += subtitle + "。"
        elif ttas == "4": level_four += subtitle + "。"
        elif ttas == "5": level_five += subtitle + "。"

        if file[page]["tables"][0][row][adjust] == "＊":
            adjust_str += subtitle + "。"

# --- 4. 迴圈結束後，處理「最後一個」主訴 ---
if dict1 is not None:
    dict1["content"] = format_content(level_one, level_two, level_three, level_four, level_five)
    dict1["remarks"] = adjust_str
    all_results.append(dict1)

#-------last one-------
dict1["content"] = format_content(level_one, level_two, level_three, level_four, level_five)
dict1["remarks"] = adjust_str
all_results.append(dict1) # 存入清單
formatted_json = json.dumps(dict1, indent=4, ensure_ascii=False)
print(f'{var} = {formatted_json}') 
# 最後呈現所有結果
print(f"總共處理了 {len(all_results)} 個標準主訴")

A0101 = {
    "metadata": {
        "category": "急診五級檢傷分類基準修正版-成人非外傷",
        "sub_category": "A01呼吸系統",
        "target": "成人"
    },
    "query_text": "呼吸短促",
    "content": "- 一級 (復甦急救)：重度呼吸窘迫(<90%)。休克。無意識(GCS3-8)。\n- 二級 (危急)：中度呼吸窘迫(<92%)。氣喘且PEFR為<40%的預期值(重度)。血行動力循環不足。意識程度改變(GCS9-13)。發燒(免疫功能缺陷)。發燒疑似敗血症，伴隨任一徵象(中度呼吸窘迫、血行動力循環不足或意識程度改變)及SIRS的條件≥3。\n- 三級 (緊急)：輕度呼吸窘迫(92-94%)。氣喘且PEFR為40-60%的預期值(中度)。血壓或心跳有異於病人之平常數值，但血行動力穩定。發燒(看起來有病容)。\n- 四級 (次緊急)：氣喘且PEFR為>60%的預期值(輕度)。發燒(看起來無病容)。無呼吸窘迫徵象。",
    "remarks": "可調節分級項目 : 無呼吸窘迫徵象。"
}
A0102 = {
    "metadata": {
        "category": "急診五級檢傷分類基準修正版-成人非外傷",
        "sub_category": "A01呼吸系統",
        "target": "成人"
    },
    "query_text": "呼吸停止",
    "content": "- 一級 (復甦急救)：呼吸停止。",
    "remarks": "可調節分級項目 : 呼吸停止。"
}
A0103 = {
    "metadata": {
        "category": "急診五級檢傷分類基準修正版-成人非外傷",
        "sub_category": "A01呼吸系統",
        "target": "成人"
    },
    "query_text": "咳嗽",
    "content": "- 二級 (危急)：中度呼吸窘迫(<92%)。血行動力循環不足。意識程度改變(GCS9-13)。發燒(免疫功能缺陷)。發燒疑似

In [35]:
print(", ".join(list_var))

A0101, A0102, A0103, A0104, A0105, A0106, A0107, A0201, A0202, A0203, A0204, A0205, A0206, A0207, A0208, A0209, A0210, A0301, A0302, A0303, A0304, A0305, A0306, A0307, A0308, A0309, A0310, A0311, A0312, A0313, A0314, A0401, A0402, A0403, A0404, A0405, A0406, A0407, A0408, A0409, A0410, A0501, A0502, A0503, A0504, A0601, A0602, A0603, A0604, A0605, A0606, A0607, A0608, A0609, A0610, A0701, A0702, A0703, A0704, A0705, A0706, A0707, A0708, A0709, A0710, A0711, A0712, A0713, A0714, A0801, A0802, A0803, A0804, A0805, A0806, A0807, A0808, A0809, A0810, A0901, A0902, A0903, A0904, A0905, A0906, A0907, A0908, A0909, A0910, A0911, A1001, A1002, A1003, A1004, A1005, A1006, A1007, A1008, A1009, A1010, A1101, A1102, A1103, A1104, A1105, A1106, A1107, A1201, A1202, A1203, A1301, A1302, A1303, A1304, A1305, A1306, A1307, A1308, A1309, A1310, A1311, A1312, A1313, A1314, A1315


In [10]:
# formatted_json = json.dumps(all_results, indent=4, ensure_ascii=False)
# print(f'{var} = {formatted_json}')  